In [1]:
import redis
import pickle
REDIS_NODE_ = "ec2-50-112-43-106.us-west-2.compute.amazonaws.com"
PASSWORD_ = "Lxs-@91q"
DB_LSH_BANDS = 5 
DB_WORD_MAP = 9
DB_TOP_1 = 10
DB_INDEX_WORDS = 0

In [2]:
vec = sqlContext.read.load("s3a://redit-preprocessed/band_band_hash_groups_40010_")
vec = vec.map(lambda x:((x[0][0],x[0][1]),x[1]))

In [3]:

def exportLSH(rdd):
    r_LSH = redis.StrictRedis(REDIS_NODE_,db=DB_LSH_BANDS,password=PASSWORD_)
    
    for item in rdd:
        r_LSH.rpush(str(item[0]),*item[1])
r_LSH = redis.StrictRedis(REDIS_NODE_,db=DB_LSH_BANDS,password=PASSWORD_)
r_LSH.flushall()
vec.foreachPartition(exportLSH)


In [4]:
print vec.first()

((4, -4754706498155953718), [232272, 225551])


In [5]:
r_word_map = redis.StrictRedis(REDIS_NODE_,db=DB_WORD_MAP,password=PASSWORD_)
dict_ = pickle.load(open( "word_map.p", 'r'))
for k,v in dict_.items():
    r_word_map.set(k,v)
    


In [3]:
posts = sqlContext.read.load("s3a://redit-preprocessed/2015_1.parquet")
posts = posts.map(lambda x:{'index':x[0],'text':x[2]}).map(lambda x:dict(x))
post_list = posts.take(1000000)

import json

In [4]:
post_list[0]

{'index': 713341,
 'text': [u'curiosity,',
  u'common',
  u'greetings',
  u'phrases',
  u'surrounding',
  u'new',
  u"year's",
  u'eve/day',
  u'japan?']}

In [6]:
import json
import os
def append_record(record):
    with open('2015_1_posts_.json', 'a') as f:
        json.dump(record, f)
        f.write(os.linesep)

# demonstrate a program writing multiple records
for post_ in post_list:
    append_record(post_)

In [7]:
my_list = []
with open('2015_1_posts_.json') as f:
    my_list = [json.loads(line) for line in f]

{u'index': 1943099,
 u'text': [u'installed',
  u'finished',
  u'mc',
  u'escher',
  u'floor.(x-post',
  u'r/woodworking)']}

In [10]:
my_list[2]

{u'index': 820136, u'text': [u'3700']}